<h4> CRYPTO TRACKER </h4>


In [1]:
import pandas as pd
import datetime as dt
import time
from binance import Client
import xlsxwriter
import xlwings as xw

In [2]:

#Clients API
api = "{your API key}"
secret = "{your API secret}"

In [3]:
client = Client(api, secret)

In [4]:
# READING THE EXCEL FILE
input_df = pd.read_excel('C:/Abdul/My Document/Fiverr/Projects/Cryptocurrency Tracker/Excel_Automation/input_template.xlsx')
input_df.info(memory_usage='deep')
input_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   coin    50 non-null     object        
 1   date    50 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 3.6 KB


,coin,date
0,ADAUSDT,2022-03-23
1,ADAUSDT,2022-03-25
2,ETHUSDT,2021-08-11
3,ETHUSDT,2021-08-14
4,BTCUSDT,2021-08-08
5,HSRBTC,2021-02-04
6,OAXETH,2022-06-02
7,DNTETH,2021-06-07
8,MCOETH,2022-03-23
9,ICNETH,2021-08-08


In [5]:
# GET HISTORICAL CRYPTO OHLC VALUES
def get_hist_crypto_data():
    history_df = pd.DataFrame()
    for i, row in input_df.iterrows():
        df_row = input_df.iloc[i]
        symbol = str(df_row['coin'])
        date = str(df_row['date'])

        # creating start date and end date
        date_temp = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        start_date = date_temp - dt.timedelta(days=1)
        end_date = date_temp + dt.timedelta(days=7)

        # converting the datetime into seconds of timestamp
        start_date_in_sec = str(
            int(time.mktime(dt.datetime.strptime(str(start_date), "%Y-%m-%d %H:%M:%S").timetuple())))
        end_date_in_sec = str(
            int(time.mktime(dt.datetime.strptime(str(end_date), "%Y-%m-%d %H:%M:%S").timetuple())))

        historical = client.get_historical_klines(symbol,
                                                  interval='1d',
                                                  start_str=start_date_in_sec,
                                                  end_str=end_date_in_sec)
        historical_df = pd.DataFrame(historical)
        historical_df['symbol'] = symbol
        history_df = pd.concat([history_df, historical_df], ignore_index=True, axis=0).drop_duplicates()

    return history_df

hist_df = get_hist_crypto_data()
hist_df

,0,1,2,3,4,5,6,7,8,9,10,11,symbol
0,1.647907e+12,0.91700000,0.98800000,0.90700000,0.97500000,172058469.40000000,1.647994e+12,164577626.02880000,280866.0,88354357.00000000,84498976.32020000,0,ADAUSDT
1,1.647994e+12,0.97500000,1.11000000,0.95700000,1.10800000,362447245.10000000,1.648080e+12,376836453.72380000,645078.0,189677928.70000000,197436607.68670000,0,ADAUSDT
2,1.648080e+12,1.10800000,1.19000000,1.07600000,1.13100000,390982460.00000000,1.648166e+12,445651286.52840000,702338.0,192422645.50000000,219300288.94500000,0,ADAUSDT
3,1.648166e+12,1.13000000,1.16700000,1.07200000,1.09600000,290456883.70000000,1.648253e+12,324802638.26710000,506738.0,140613667.10000000,157248129.85860000,0,ADAUSDT
4,1.648253e+12,1.09600000,1.16000000,1.07900000,1.15200000,143117986.10000000,1.648339e+12,161024339.18000000,271206.0,75325602.00000000,84777494.22780000,0,ADAUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1.612656e+12,0.00012510,0.00012770,0.00011400,0.00011930,2325376.53000000,1.612742e+12,279.73482168,23680.0,1075597.54000000,129.45485986,0,OMGBTC
82,1.612742e+12,0.00011940,0.00012770,0.00010210,0.00010490,3215604.10000000,1.612829e+12,368.74653082,31702.0,1470566.32000000,167.96128796,0,OMGBTC
83,1.612829e+12,0.00010490,0.00011250,0.00010000,0.00011000,2758950.39000000,1.612915e+12,293.53937449,28750.0,1337441.40000000,142.31145139,0,OMGBTC
84,1.612915e+12,0.00011020,0.00012750,0.00010860,0.00012460,4266072.69000000,1.613002e+12,503.51593342,47339.0,2078812.96000000,245.71468026,0,OMGBTC


In [6]:
hist_df.drop_duplicates()

,0,1,2,3,4,5,6,7,8,9,10,11,symbol
0,1.647907e+12,0.91700000,0.98800000,0.90700000,0.97500000,172058469.40000000,1.647994e+12,164577626.02880000,280866.0,88354357.00000000,84498976.32020000,0,ADAUSDT
1,1.647994e+12,0.97500000,1.11000000,0.95700000,1.10800000,362447245.10000000,1.648080e+12,376836453.72380000,645078.0,189677928.70000000,197436607.68670000,0,ADAUSDT
2,1.648080e+12,1.10800000,1.19000000,1.07600000,1.13100000,390982460.00000000,1.648166e+12,445651286.52840000,702338.0,192422645.50000000,219300288.94500000,0,ADAUSDT
3,1.648166e+12,1.13000000,1.16700000,1.07200000,1.09600000,290456883.70000000,1.648253e+12,324802638.26710000,506738.0,140613667.10000000,157248129.85860000,0,ADAUSDT
4,1.648253e+12,1.09600000,1.16000000,1.07900000,1.15200000,143117986.10000000,1.648339e+12,161024339.18000000,271206.0,75325602.00000000,84777494.22780000,0,ADAUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1.612656e+12,0.00012510,0.00012770,0.00011400,0.00011930,2325376.53000000,1.612742e+12,279.73482168,23680.0,1075597.54000000,129.45485986,0,OMGBTC
82,1.612742e+12,0.00011940,0.00012770,0.00010210,0.00010490,3215604.10000000,1.612829e+12,368.74653082,31702.0,1470566.32000000,167.96128796,0,OMGBTC
83,1.612829e+12,0.00010490,0.00011250,0.00010000,0.00011000,2758950.39000000,1.612915e+12,293.53937449,28750.0,1337441.40000000,142.31145139,0,OMGBTC
84,1.612915e+12,0.00011020,0.00012750,0.00010860,0.00012460,4266072.69000000,1.613002e+12,503.51593342,47339.0,2078812.96000000,245.71468026,0,OMGBTC


In [11]:
hist_df.info(memory_usage="deep")


<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 0 to 85
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       86 non-null     float64
 1   1       86 non-null     object 
 2   2       86 non-null     object 
 3   3       86 non-null     object 
 4   4       86 non-null     object 
 5   5       86 non-null     object 
 6   6       86 non-null     float64
 7   7       86 non-null     object 
 8   8       86 non-null     float64
 9   9       86 non-null     object 
 10  10      86 non-null     object 
 11  11      86 non-null     object 
 12  symbol  86 non-null     object 
dtypes: float64(3), object(10)
memory usage: 59.9 KB


In [12]:
def process_data():
    # rename columns
    hist_df.columns = (['opentime',
                        'Open',
                        'High',
                        'Low',
                        'Close',
                        'Volume',
                        'Close time',
                        'Quote asset volume',
                        'Number of trades',
                        'TB base asset volume',
                        'TB quote asset volume',
                        'Unused field, ignore',
                        'symbol'])

    # convert the open time to date
    hist_df['opentime'] = pd.to_datetime(hist_df['opentime'] / 1000, unit='s')

    # select the column to convert into numeric type
    # to_numeric = ['Open', 'High', 'Low', 'Close']

    # convert the object type to numerics
    # hist_df[to_numeric] = hist_df[to_numeric].apply(pd.to_numeric, axis=1)


process_data()
hist_df

,opentime,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,TB base asset volume,TB quote asset volume,"Unused field, ignore",symbol
0,2022-03-22,0.91700000,0.98800000,0.90700000,0.97500000,172058469.40000000,1.647994e+12,164577626.02880000,280866.0,88354357.00000000,84498976.32020000,0,ADAUSDT
1,2022-03-23,0.97500000,1.11000000,0.95700000,1.10800000,362447245.10000000,1.648080e+12,376836453.72380000,645078.0,189677928.70000000,197436607.68670000,0,ADAUSDT
2,2022-03-24,1.10800000,1.19000000,1.07600000,1.13100000,390982460.00000000,1.648166e+12,445651286.52840000,702338.0,192422645.50000000,219300288.94500000,0,ADAUSDT
3,2022-03-25,1.13000000,1.16700000,1.07200000,1.09600000,290456883.70000000,1.648253e+12,324802638.26710000,506738.0,140613667.10000000,157248129.85860000,0,ADAUSDT
4,2022-03-26,1.09600000,1.16000000,1.07900000,1.15200000,143117986.10000000,1.648339e+12,161024339.18000000,271206.0,75325602.00000000,84777494.22780000,0,ADAUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,2021-02-07,0.00012510,0.00012770,0.00011400,0.00011930,2325376.53000000,1.612742e+12,279.73482168,23680.0,1075597.54000000,129.45485986,0,OMGBTC
82,2021-02-08,0.00011940,0.00012770,0.00010210,0.00010490,3215604.10000000,1.612829e+12,368.74653082,31702.0,1470566.32000000,167.96128796,0,OMGBTC
83,2021-02-09,0.00010490,0.00011250,0.00010000,0.00011000,2758950.39000000,1.612915e+12,293.53937449,28750.0,1337441.40000000,142.31145139,0,OMGBTC
84,2021-02-10,0.00011020,0.00012750,0.00010860,0.00012460,4266072.69000000,1.613002e+12,503.51593342,47339.0,2078812.96000000,245.71468026,0,OMGBTC


In [9]:
hist_df = hist_df.drop(columns=['Volume','Close time', 'Quote asset volume','Number of trades',
                                'TB base asset volume', 'TB quote asset volume', 'Unused field, ignore'])
hist_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 0 to 85
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   opentime  86 non-null     datetime64[ns]
 1   Open      86 non-null     object        
 2   High      86 non-null     object        
 3   Low       86 non-null     object        
 4   Close     86 non-null     object        
 5   symbol    86 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 4.7+ KB


In [10]:
hist_df

,opentime,Open,High,Low,Close,symbol
0,2022-03-22,0.91700000,0.98800000,0.90700000,0.97500000,ADAUSDT
1,2022-03-23,0.97500000,1.11000000,0.95700000,1.10800000,ADAUSDT
2,2022-03-24,1.10800000,1.19000000,1.07600000,1.13100000,ADAUSDT
3,2022-03-25,1.13000000,1.16700000,1.07200000,1.09600000,ADAUSDT
4,2022-03-26,1.09600000,1.16000000,1.07900000,1.15200000,ADAUSDT
...,...,...,...,...,...,...
81,2021-02-07,0.00012510,0.00012770,0.00011400,0.00011930,OMGBTC
82,2021-02-08,0.00011940,0.00012770,0.00010210,0.00010490,OMGBTC
83,2021-02-09,0.00010490,0.00011250,0.00010000,0.00011000,OMGBTC
84,2021-02-10,0.00011020,0.00012750,0.00010860,0.00012460,OMGBTC


In [11]:
def add_yeterday_date_columns():
    concat_df = pd.DataFrame()
    for i , row in input_df.iterrows():
        input_df_row = input_df.loc[i]
        date = str(input_df_row['date'])
        date_temp = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        yesterday = date_temp - dt.timedelta(days=1)

        new_date_series = pd.Series([yesterday])
        concat_df = pd.concat([concat_df, new_date_series], ignore_index=True, axis=0)
        input_df['Yesterday'] = concat_df[0]
    return input_df

def add_date_columns(days_to_add, column_name):
    concat_df = pd.DataFrame()
    for i , row in input_df.iterrows():
        input_df_row = input_df.loc[i]
        date = str(input_df_row['date'])
        date_temp = dt.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        new_date = date_temp + dt.timedelta(days=days_to_add)

        new_date_series = pd.Series([new_date])
        concat_df = pd.concat([concat_df, new_date_series], ignore_index=True, axis=0)
        input_df[column_name] = concat_df[0]
    return input_df


add_yeterday_date_columns()

add_date_columns(1, 'day 1')
add_date_columns(2, 'day 2')
add_date_columns(3, 'day 3')
add_date_columns(4, 'day 4')
add_date_columns(5, 'day 5')
add_date_columns(6, 'day 6')
add_date_columns(7, 'day 7')
input_df.info()
input_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   coin       50 non-null     object        
 1   date       50 non-null     datetime64[ns]
 2   Yesterday  50 non-null     datetime64[ns]
 3   day 1      50 non-null     datetime64[ns]
 4   day 2      50 non-null     datetime64[ns]
 5   day 3      50 non-null     datetime64[ns]
 6   day 4      50 non-null     datetime64[ns]
 7   day 5      50 non-null     datetime64[ns]
 8   day 6      50 non-null     datetime64[ns]
 9   day 7      50 non-null     datetime64[ns]
dtypes: datetime64[ns](9), object(1)
memory usage: 4.0+ KB


,coin,date,Yesterday,day 1,day 2,day 3,day 4,day 5,day 6,day 7
0,ADAUSDT,2022-03-23,2022-03-22,2022-03-24,2022-03-25,2022-03-26,2022-03-27,2022-03-28,2022-03-29,2022-03-30
1,ADAUSDT,2022-03-25,2022-03-24,2022-03-26,2022-03-27,2022-03-28,2022-03-29,2022-03-30,2022-03-31,2022-04-01
2,ETHUSDT,2021-08-11,2021-08-10,2021-08-12,2021-08-13,2021-08-14,2021-08-15,2021-08-16,2021-08-17,2021-08-18
3,ETHUSDT,2021-08-14,2021-08-13,2021-08-15,2021-08-16,2021-08-17,2021-08-18,2021-08-19,2021-08-20,2021-08-21
4,BTCUSDT,2021-08-08,2021-08-07,2021-08-09,2021-08-10,2021-08-11,2021-08-12,2021-08-13,2021-08-14,2021-08-15
5,HSRBTC,2021-02-04,2021-02-03,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,2021-02-11
6,OAXETH,2022-06-02,2022-06-01,2022-06-03,2022-06-04,2022-06-05,2022-06-06,2022-06-07,2022-06-08,2022-06-09
7,DNTETH,2021-06-07,2021-06-06,2021-06-08,2021-06-09,2021-06-10,2021-06-11,2021-06-12,2021-06-13,2021-06-14
8,MCOETH,2022-03-23,2022-03-22,2022-03-24,2022-03-25,2022-03-26,2022-03-27,2022-03-28,2022-03-29,2022-03-30
9,ICNETH,2021-08-08,2021-08-07,2021-08-09,2021-08-10,2021-08-11,2021-08-12,2021-08-13,2021-08-14,2021-08-15


In [12]:

def merge_data(df_name, input_df_date):
    # merge crypto data with input data
    df_name = pd.DataFrame()
    df_name = pd.concat([df_name, input_df], axis=1, ignore_index=True)
    df_name.columns = (['coin', 'day 0', 'Yesterday', 'day 1', 'day 2', 'day 3', 'day 4',
                        'day 5', 'day 6', 'day 7'])
    df_name = df_name[['coin', input_df_date]]


    # df_name = pd.concat([df_name, hist_df], join='inner', axis=1, ignore_index=True, copy=False)
    df_name = pd.merge(left=df_name,right=hist_df, left_on=['coin', input_df_date], right_on=['symbol', 'opentime'],
                       how='left', copy=False)

    df_name.columns = ([['coin', input_df_date, 'opentime', 'Open', 'High', 'Low', 'Close', 'symbol']])

    df_name = df_name[['Open', 'High', 'Low', 'Close']]

    return df_name

yesterday_df = merge_data('yesterday_df', 'Yesterday')
day0_df = merge_data('day0_df', 'day 0')
day1_df = merge_data('day1_df', 'day 1')
day2_df = merge_data('day2_df', 'day 2')
day3_df = merge_data('day3_df', 'day 3')
day4_df = merge_data('day4_df', 'day 4')
day5_df = merge_data('day5_df', 'day 5')
day6_df = merge_data('day6_df', 'day 6')
day7_df = merge_data('day7_df', 'day 7')


yesterday_df.info()
yesterday_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   (Open,)   11 non-null     object
 1   (High,)   11 non-null     object
 2   (Low,)    11 non-null     object
 3   (Close,)  11 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


,Open,High,Low,Close
0,0.91700000,0.98800000,0.90700000,0.97500000
1,1.10800000,1.19000000,1.07600000,1.13100000
2,3160.51000000,3233.00000000,3051.22000000,3141.66000000
3,3046.11000000,3327.83000000,3033.80000000,3322.32000000
4,42836.87000000,44700.00000000,42446.41000000,44572.54000000
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [13]:
yesterday_df

,Open,High,Low,Close
0,0.91700000,0.98800000,0.90700000,0.97500000
1,1.10800000,1.19000000,1.07600000,1.13100000
2,3160.51000000,3233.00000000,3051.22000000,3141.66000000
3,3046.11000000,3327.83000000,3033.80000000,3322.32000000
4,42836.87000000,44700.00000000,42446.41000000,44572.54000000
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [14]:
def final_frame():
    final_df = pd.concat([yesterday_df,
                          input_df[['coin', 'date']],
                          day0_df,
                          day1_df,
                          day2_df,
                          day3_df,
                          day4_df,
                          day5_df,
                          day6_df,
                          day7_df], axis=1, ignore_index=True)

    # Renaming the columns
    final_df.columns = (['Open', 'High', 'Low', 'Close',
                         'coin', 'date',
                         'Open0', 'High0', 'Low0', 'Close0',
                         'Open1', 'High1', 'Low1', 'Close1',
                         'Open2', 'High2', 'Low2', 'Close2',
                         'Open3', 'High3', 'Low3', 'Close3',
                         'Open4', 'High4', 'Low4', 'Close4',
                         'Open5', 'High5', 'Low5', 'Close5',
                         'Open6', 'High6', 'Low6', 'Close6',
                         'Open7', 'High7', 'Low7', 'Close7'])
    return final_df

final_df = final_frame()
final_df.info()
final_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Open    11 non-null     object        
 1   High    11 non-null     object        
 2   Low     11 non-null     object        
 3   Close   11 non-null     object        
 4   coin    50 non-null     object        
 5   date    50 non-null     datetime64[ns]
 6   Open0   11 non-null     object        
 7   High0   11 non-null     object        
 8   Low0    11 non-null     object        
 9   Close0  11 non-null     object        
 10  Open1   11 non-null     object        
 11  High1   11 non-null     object        
 12  Low1    11 non-null     object        
 13  Close1  11 non-null     object        
 14  Open2   11 non-null     object        
 15  High2   11 non-null     object        
 16  Low2    11 non-null     object        
 17  Close2  11 non-null     object        
 18  Open3   11 n

,Open,High,Low,Close,coin,date,Open0,High0,Low0,Close0,...,Low5,Close5,Open6,High6,Low6,Close6,Open7,High7,Low7,Close7
0,0.91700000,0.98800000,0.90700000,0.97500000,ADAUSDT,2022-03-23,0.97500000,1.11000000,0.95700000,1.10800000,...,1.15700000,1.17000000,1.17000000,1.24300000,1.16600000,1.19100000,1.19100000,1.21800000,1.16100000,1.18900000
1,1.10800000,1.19000000,1.07600000,1.13100000,ADAUSDT,2022-03-25,1.13000000,1.16700000,1.07200000,1.09600000,...,1.16100000,1.18900000,1.18900000,1.22800000,1.13300000,1.14100000,1.14100000,1.17500000,1.10400000,1.16500000
2,3160.51000000,3233.00000000,3051.22000000,3141.66000000,ETHUSDT,2021-08-11,3141.14000000,3274.00000000,3120.27000000,3160.50000000,...,3135.22000000,3146.53000000,3147.18000000,3290.00000000,2991.89000000,3011.72000000,3011.73000000,3125.81000000,2950.00000000,3014.55000000
3,3046.11000000,3327.83000000,3033.80000000,3322.32000000,ETHUSDT,2021-08-14,3322.33000000,3330.98000000,3200.00000000,3263.16000000,...,2958.33000000,3184.12000000,3184.11000000,3300.00000000,3177.16000000,3284.82000000,3284.81000000,3309.16000000,3207.00000000,3223.43000000
4,42836.87000000,44700.00000000,42446.41000000,44572.54000000,BTCUSDT,2021-08-08,44572.54000000,45310.00000000,43261.00000000,43794.37000000,...,44217.39000000,47800.00000000,47799.99000000,48144.00000000,45971.03000000,47068.51000000,47068.50000000,47372.27000000,45500.00000000,46973.82000000
5,NaN,NaN,NaN,NaN,HSRBTC,2021-02-04,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,OAXETH,2022-06-02,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,DNTETH,2021-06-07,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,MCOETH,2022-03-23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,ICNETH,2021-08-08,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# def merge_data():
#     # merge crypto data with input data
#     df1 = pd.merge(left=input_df,right=hist_df, left_on=['coin', 'date'], right_on=['symbol', 'opentime'], how='left')
#     df2 = pd.merge(left=df1, right=hist_df, left_on=['coin', 'Yesterday'], right_on=['symbol', 'opentime'], how='left',                               suffixes=['_day0', '_yesterday'])
#     df3 = pd.merge(left=df2,right=hist_df, left_on=['coin', 'day 1'], right_on=['symbol', 'opentime'], how='left')
#     df4 = pd.merge(left=df3,right=hist_df, left_on=['coin', 'day 2'], right_on=['symbol', 'opentime'], how='left',
#                    suffixes=['_day1', '_day2'])
#     df5 = pd.merge(left=df4,right=hist_df, left_on=['coin', 'day 3'], right_on=['symbol', 'opentime'], how='left')
#     df6 = pd.merge(left=df5,right=hist_df, left_on=['coin', 'day 4'], right_on=['symbol', 'opentime'], how='left',
#                    suffixes=['_day3', '_day4'])
#     df7 = pd.merge(left=df6,right=hist_df, left_on=['coin', 'day 5'], right_on=['symbol', 'opentime'], how='left')
#     df8 = pd.merge(left=df7,right=hist_df, left_on=['coin', 'day 6'], right_on=['symbol', 'opentime'], how='left',
#                    suffixes=['_day5', '_day6'])
#     gc.collect()
#     df9 = pd.merge(left=df8,right=hist_df, left_on=['coin', 'day 7'], right_on=['symbol', 'opentime'], how='left')
#
#     # df9 = df9.drop(columns=['symbol_day0', 'opentime_day0', 'symbol_yesterday', 'opentime_yesterday', 'symbol_day1',
#     #                   'opentime_day1', 'symbol_day2', 'opentime_day2', 'symbol_day3', 'opentime_day3',
#     #                   'symbol_day4', 'opentime_day4', 'symbol_day5', 'opentime_day5', 'symbol_day6',
#     #                   'opentime_day6', 'symbol', 'opentime'])
#
#     # rearranging the order of columns and calling only the necessary columns
#     df9 = df9[[ 'Open_yesterday', 'High_yesterday', 'Low_yesterday', 'Close_yesterday',
#                 'coin','date', 'Open_day0', 'High_day0', 'Low_day0', 'Close_day0',
#                 'Open_day1', 'High_day1', 'Low_day1', 'Close_day1',
#                 'Open_day2', 'High_day2', 'Low_day2', 'Close_day2',
#                 'Open_day3', 'High_day3', 'Low_day3', 'Close_day3',
#                 'Open_day4', 'High_day4', 'Low_day4', 'Close_day4',
#                 'Open_day5', 'High_day5', 'Low_day5', 'Close_day5',
#                 'Open_day6', 'High_day6', 'Low_day6', 'Close_day6',
#                 'Open', 'High', 'Low', 'Close']]
#     return df9
#
# final_df = merge_data()
# final

In [16]:
def save_to_excel():
    with xw.App(visible=False) as app:
        wb = app.books.open('Crypto_Tracker.xlsx')

        wb.sheets('CryptoTracker').range('A3').value = final_df
        wb.sheets['CryptoTracker'].range('3:3').delete()
        wb.save("C:\Abdul\My Document\Fiverr\Projects\Cryptocurrency Tracker\Excel_Automation/CryptoTracker.xlsx")

save_to_excel()

In [17]:
# output_file = """C:\Abdul\My Document\Fiverr\Projects/test.xlsx"""
# with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
#     # convert the df to xlsxwriter object
#     final_df.to_excel(writer, sheet_name='CryptoTracker')
#
#     # Add document properties
#     workbook = writer.book
#     workbook.set_properties(
#         {
#             'title': 'Crypto Tracker',
#             'subject': 'Crypto prices for previous and past days',
#             'author': 'Roshan from Avion Analytics',
#             'Manager': 'None',
#             'company': 'Avion Analytics',
#             'category': 'cryptocurrency',
#             'keywords': 'Pandas, XlsxWriter',
#             'comment': 'If you need further improvements or software developments search for us online',
#             'status': 'Final',
#         }
#     )

<h5>TEST</h5>

In [18]:
# url = 'https://api.binance.com'
#
# symbol = 'btcusdt'
# interval = '1d'
# startTime = str(int(dt.datetime(2020,5,1).timestamp() * 1000))
# endTime = str(int(dt.datetime(2020,6,1).timestamp() * 1000))
# limit = '100'
#
# req_params = {'symbol': symbol, 'interval': interval, 'startTime': startTime, 'endTime': endTime}
#
# response = requests.get(url, params= req_params)

In [19]:
# history_df['opentime'] = [dt.datetime.fromtimestamp(x/ 1000.0) for x in history_df.opentime]

# history_df['opentime'] = pd.to_datetime(history_df['opentime']/1000, unit='s')
# history_df['Close time'] = pd.to_datetime(history_df['Close time']/1000,unit='s')
# history_df['open_time'] = history_df['open time'] // (24 * 3600)


# for i, row in history_df.iterrows():
#     index = history_df.loc[i]
#     history_df[i, 'open time'] = dt.datetime.fromtimestamp(index['open time'] / 1e3)

In [20]:
# tickers = client.get_all_tickers()
# ticker_df = pd.DataFrame(tickers)
# ticker_df

In [21]:
# ticker_df.to_excel("C:\Abdul\My Document\Fiverr\Projects\Cryptocurrency Tracker\Excel_Automation/All_Coins.xlsx")

In [22]:
# crypto_df = pd.DataFrame(tickers)
# crypto_df.head()

In [23]:
# startTime = str(int(dt.datetime(2020,5,1).timestamp() * 1000))
# endTime = str(int(dt.datetime(2020,5,2).timestamp() * 1000))
# historical = client.get_historical_klines('BTCUSDT',
#                                           interval='1d',
#                                           start_str=startTime,
#                                           end_str=endTime)

In [24]:
# historical_df = pd.DataFrame(historical)
# historical_df

In [25]:
# historical_df.columns = (['open time',
#                          'Open',
#                          'High',
#                          'Low',
#                          'Close',
#                          'Volume',
#                          'Close time',
#                          'Quote asset volume',
#                          'Number of trades',
#                          'Taker buy base asset volume',
#                          'Taker buy quote asset volume',
#                          'Unused field, ignore.'])
# historical_df.head()

In [26]:
# historical_df.dtypes

In [27]:
# historical_df['open time'] = pd.to_datetime(historical_df['open time']/1000,unit='s')
# historical_df['Close time'] = pd.to_datetime(historical_df['Close time']/1000,unit='s')

In [28]:
# historical_df

In [29]:
# to_numeric = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume', 'Number of trades',
#